In [1]:
import sys
import os
import dill
import matplotlib.pyplot as plt

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.qpy import load as qpy_load, dump as qpy_dump

# Add the path to the parent of the parent directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

# Now you can import your modules
from circuit_constructors import construct_A, construct_Q

### 1 Construct A and Q for all Pauli strings

In [2]:
# Load decomp and optim_res
with open('../pauli_decomp/sorted_pauli_dict.pkl', 'rb') as f:
    sorted_pauli_dict = dill.load(f)
with open('../optim/optim_opt_res.pkl', 'rb') as f:
    opt_res = dill.load(f)

In [3]:
# Extract opt_params
opt_params = opt_res.x

# Set l
l = 3

# Create the main folder if it doesn't exist
if not os.path.exists('circuits'):
    os.makedirs('circuits')

# Traverse the Pauli strings
for i, (pauli_string, coeff) in enumerate(sorted_pauli_dict.items()):
    # Create a folder for this Pauli string
    folder_path = os.path.join('circuits', str(i))
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Construct A
    A = construct_A(l, opt_params, pauli_string)
    
    # Construct Q
    Q = construct_Q(A)
    
    # Save A
    with open(os.path.join(folder_path, 'circuit_A.qpy'), 'wb') as f:
        qpy_dump(A, f)
    
    # Save Q
    with open(os.path.join(folder_path, 'circuit_Q.qpy'), 'wb') as f:
        qpy_dump(Q, f)
    
    print(f"Processed Pauli string {i}: {pauli_string}")

print("All circuits generated and saved.")

Processed Pauli string 0: IIII
Processed Pauli string 1: ZIZI
Processed Pauli string 2: ZIIZ
Processed Pauli string 3: IZZI
Processed Pauli string 4: IZIZ
Processed Pauli string 5: IIIZ


Processed Pauli string 6: IZII
Processed Pauli string 7: ZZII
Processed Pauli string 8: IIZZ
Processed Pauli string 9: IIZI
Processed Pauli string 10: ZIII
Processed Pauli string 11: XXXX


Processed Pauli string 12: XXYY
Processed Pauli string 13: YYXX
Processed Pauli string 14: YYYY
All circuits generated and saved.


### 2 Transpilation

In [4]:
# Create an instance of the AerSimulator
aer_sim = AerSimulator()

# Dictionary to store depths of transpiled circuits
transpiled_depths = {}

# Walk through the numbered folders in circuits
for folder in os.listdir('circuits'):
    folder_path = os.path.join('circuits', folder)
    if os.path.isdir(folder_path):
        # Process circuit_A.qpy and circuit_Q.qpy
        for circuit_file in ['circuit_A.qpy', 'circuit_Q.qpy']:
            input_path = os.path.join(folder_path, circuit_file)
            if os.path.exists(input_path):
                # Load the circuit
                with open(input_path, 'rb') as f:
                    circuit = qpy_load(f)[0]
                
                # Transpile the circuit
                transpiled_circuit = transpile(circuit, backend=aer_sim, optimization_level=3)
                
                # Save the transpiled circuit
                transpiled_output_path = os.path.join(folder_path, f'transpiled_{circuit_file}')
                
                with open(transpiled_output_path, 'wb') as f:
                    qpy_dump(transpiled_circuit, f)
                
                print(f"Transpiled and saved: {transpiled_output_path}")
                
                # Store the depth of the transpiled circuit
                circuit_key = f"{folder}_{circuit_file.split('.')[0]}"
                transpiled_depths[circuit_key] = transpiled_circuit.depth()

print("All circuits have been transpiled and saved.")

# Print depths for both circuit A and Q
for folder in sorted([d for d in os.listdir('circuits') if os.path.isdir(os.path.join('circuits', d))], key=int):
    circuit_A_key = f"{folder}_circuit_A"
    circuit_Q_key = f"{folder}_circuit_Q"
    if circuit_A_key in transpiled_depths and circuit_Q_key in transpiled_depths:
        print(f"For the {int(folder) + 1}th Pauli string:")
        print(f"  The depth of circuit A is {transpiled_depths[circuit_A_key]}")
        print(f"  The depth of circuit Q is {transpiled_depths[circuit_Q_key]}")



Transpiled and saved: circuits/0/transpiled_circuit_A.qpy


Transpiled and saved: circuits/0/transpiled_circuit_Q.qpy


Transpiled and saved: circuits/1/transpiled_circuit_A.qpy


Transpiled and saved: circuits/1/transpiled_circuit_Q.qpy


Transpiled and saved: circuits/10/transpiled_circuit_A.qpy


...

Transpiled and saved: circuits/6/transpiled_circuit_Q.qpy
Transpiled and saved: circuits/7/transpiled_circuit_A.qpy


Transpiled and saved: circuits/7/transpiled_circuit_Q.qpy


Transpiled and saved: circuits/8/transpiled_circuit_A.qpy


Transpiled and saved: circuits/8/transpiled_circuit_Q.qpy
Transpiled and saved: circuits/9/transpiled_circuit_A.qpy


Transpiled and saved: circuits/9/transpiled_circuit_Q.qpy
All circuits have been transpiled and saved.
For the 1th Pauli string:
  The depth of circuit A is 17
  The depth of circuit Q is 35
For the 2th Pauli string:
  The depth of circuit A is 19
  The depth of circuit Q is 40
For the 3th Pauli string:
  The depth of circuit A is 19
  The depth of circuit Q is 40
For the 4th Pauli string:
  The depth of circuit A is 19
  The depth of circuit Q is 40
For the 5th Pauli string:
  The depth of circuit A is 19
  The depth of circuit Q is 40
For the 6th Pauli string:
  The depth of circuit A is 19
  The depth of circuit Q is 39
For the 7th Pauli string:
  The depth of circuit A is 18
  The depth of circuit Q is 37
For the 8th Pauli string:
  The depth of circuit A is 18
  The depth of circuit Q is 38
For the 9th Pauli string:
  The depth of circuit A is 20
  The depth of circuit Q is 41
For the 10th Pauli string:
  The depth of circuit A is 19
  The depth of circuit Q is 39
For the 11th Pau

### 3 Plot all circuits

In [5]:
# # Walk through the numbered folders in circuits
# for folder in os.listdir('circuits'):
#     folder_path = os.path.join('circuits', folder)
#     if os.path.isdir(folder_path):
#         # Process all circuit files in the folder
#         for circuit_file in os.listdir(folder_path):
#             if circuit_file.endswith('.qpy'):
#                 input_path = os.path.join(folder_path, circuit_file)
                
#                 # Load the circuit
#                 with open(input_path, 'rb') as f:
#                     circuit = qpy_load(f)[0]
                
#                 # Draw the circuit
#                 circuit_img = circuit.draw(output='mpl', style={
#                     'dpi': 150,
#                     'font_size': 20,
#                     'backgroundcolor': '#EEEEEE'
#                 })
#                 circuit_img.figure.set_size_inches(30, 15)
                
#                 # Save the plot
#                 output_filename = f"{os.path.splitext(circuit_file)[0]}.png"
#                 output_path = os.path.join(folder_path, output_filename)
#                 circuit_img.figure.savefig(output_path, dpi=300, bbox_inches='tight')
#                 plt.close(circuit_img.figure)
                
#                 print(f"Saved circuit plot: {output_path}")

# print("All circuit plots have been generated and saved.")